---
title: "Lab 4"
format:
  html:
    embed-resources: true
    code-fold: true
execute:
  echo: true
  warning: false
  message: false
---

# **Part 1**

In [1]:
# I wrote all of this code out based on my week 4 notes, which included a similar format
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://tastesbetterfromscratch.com/meal-plan-191/" #Trying out plan 191
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

req = requests.Session()
response = req.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")


In [2]:
#I had chat GPT refine parts of this, I identified whcih aspects needed to be pulled, but chat was useful in organizing it in a more efficient/useful way as I was getting a few errors, but I know
# what each of these components is doing and how they contribute to correctly pulling it

# Initalize an empty list for the data to be put in
meal_plan_data = []

# This seems to be where most the content is locate
main_content = soup.find('div', class_='site-inner')

if main_content:
    # Based on the source code using source-code: the meal plan items are in <p> tags with class 'has-text-align-left'
    meal_plan_items = main_content.find_all('p', class_='has-text-align-left')

    for item in meal_plan_items:
        text = item.get_text(strip=True)
        parts = text.split(':')
        if len(parts) >= 2:
            day_of_week = parts[0].strip()
            recipe_info = parts[1].strip()

            # I am Spliting the recipe info to separate name and price
            name_and_price_parts = recipe_info.rsplit('$', 1)
            if len(name_and_price_parts) == 2:
                name_of_recipe = name_and_price_parts[0].strip()
                price_of_recipe = '$' + name_and_price_parts[1].strip()
            else:
                name_of_recipe = recipe_info.strip()
                price_of_recipe = 'N/A'


            # Look for an anchor tag (<a>) within the current item
            link_to_recipe = 'N/A'
            link_tag = item.find('a')
            if link_tag and link_tag.get('href'): # THis was what was used
                 link_to_recipe = link_tag.get('href')


            meal_plan_data.append({
                'Day of the Week': day_of_week,
                'Name of Recipe': name_of_recipe,
                'Link to Recipe': link_to_recipe,
                'Price of Recipe': price_of_recipe
            })

# Creating a pandas DataFrame from the extracted data
df_meal_plan = pd.DataFrame(meal_plan_data, columns=['Day of the Week', 'Name of Recipe', 'Link to Recipe', 'Price of Recipe'])

# Displaying the dataframe to make sure this is correctly pulling
display(df_meal_plan)

,Day of the Week,Name of Recipe,Link to Recipe,Price of Recipe
0,Monday,Chicken Tinga,https://tastesbetterfromscratch.com/chicken-ti...,$10.22
1,Tuesday,BBQ Chicken Pizza,https://tastesbetterfromscratch.com/barbeque-c...,$10.26
2,Wednesday,Hamburger Gravy,https://tastesbetterfromscratch.com/hamburger-...,$8.20
3,Thursday,Manicotti,https://tastesbetterfromscratch.com/manicotti/,$17.63
4,Friday,Slow Cooker Ribs,https://tastesbetterfromscratch.com/slow-cooke...,$24.42


This aligsn with the info from the website!

# **Part 2**

In [3]:
# Again, for this part I used a similar strategy in last weeks notes
url = "https://tasty.p.rapidapi.com/recipes/list"

querystring = {"from":"0","size":"20","q":"Chicken Tinga"} #Just using this as an example as it was Monday's for 191

headers = {
    "x-rapidapi-key": "10bff72081msh0d8b0bbdb700e4ap1d07bfjsn04ddcf3e4d60", #MY API CODE
    "x-rapidapi-host": "tasty.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

monday_recipes = pd.json_normalize(response.json(), "results")
print(monday_recipes)
print(f'The number of matching recipes is: {len(monday_recipes)}')

   approved_at aspect_ratio  \
0   1498744377          1:1   
1   1624981831         16:9   
2   1610462731          1:1   
3   1718992824         9:16   
4   1666288749          1:1   
5   1651852968         9:16   
6   1634567313          1:1   

                                          beauty_url  brand  brand_id buzz_id  \
0                                               None    NaN       NaN    None   
1                                               None    NaN       NaN    None   
2                                               None    NaN      38.0    None   
3                                               None    NaN      89.0    None   
4  https://img.buzzfeed.com/video-api-prod/assets...    NaN       3.0    None   
5                                               None    NaN       NaN    None   
6  https://img.buzzfeed.com/video-api-prod/assets...    NaN      33.0    None   

  canonical_id                                       compilations  \
0   recipe:639  [{'approved_at': 

In [4]:
# Chat advised me to set these options
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 150)

print(monday_recipes.columns.tolist())   #I wanted to see what all the columns would look like
print(monday_recipes.iloc[0].to_string())

['approved_at', 'aspect_ratio', 'beauty_url', 'brand', 'brand_id', 'buzz_id', 'canonical_id', 'compilations', 'cook_time_minutes', 'country', 'created_at', 'credits', 'description', 'draft_status', 'facebook_posts', 'id', 'inspired_by_url', 'instructions', 'is_app_only', 'is_one_top', 'is_shoppable', 'is_subscriber_content', 'keywords', 'language', 'name', 'num_servings', 'nutrition_visibility', 'original_video_url', 'prep_time_minutes', 'promotion', 'renditions', 'sections', 'seo_path', 'seo_title', 'servings_noun_plural', 'servings_noun_singular', 'show_id', 'slug', 'tags', 'thumbnail_alt_text', 'thumbnail_url', 'tips_and_ratings_enabled', 'topics', 'total_time_minutes', 'updated_at', 'video_ad_content', 'video_id', 'video_url', 'yields', 'nutrition.calories', 'nutrition.carbohydrates', 'nutrition.fat', 'nutrition.fiber', 'nutrition.protein', 'nutrition.sugar', 'nutrition.updated_at', 'price.consumption_portion', 'price.consumption_total', 'price.portion', 'price.total', 'price.updat

In [5]:
summary = monday_recipes[[
    "name",
    "user_ratings.score",
    "nutrition.calories",
    "nutrition.protein",
    "nutrition.fat",
    "nutrition.carbohydrates"
]]

print(summary.to_string(index=False))


                                name  user_ratings.score  nutrition.calories  nutrition.protein  nutrition.fat  nutrition.carbohydrates
Chicken Tinga Tacos (Tinga De Pollo)            0.910112               653.0               39.0           33.0                     56.0
                   Tostadas De Tinga            1.000000               133.0               11.0            2.0                     16.0
  Chicken Tinga Sheet Pan Quesadilla            0.764706               390.0               20.0           18.0                     31.0
          Tostadas De Tinga De Pollo            0.916667               287.0               27.0           16.0                     10.0
    Skillet Chicken Tinga Enchiladas            0.900000              1219.0               66.0           72.0                     75.0
   Bella’s Chicken Tinga Party Board            1.000000                52.0                7.0            2.0                      0.0
         Chicken Tinga Loaded Nachos            

I checked this on the general website and there was also 7 total chicken tanga recepies, so I feel good about this result

I also went back later and made this summary so I could see that it had these nutrtion facts that I later will need

This is the info that seemed most relevant to keep for this assignment

# **Part 3**

In [6]:
# I want to state that PART 3 is naturally where I used the most AI support being relatively new to function making
#. I always started with my idea, but I often needed some AI suppliment to get me through some of the smaller issues I
# was havign and help around issues I was encountering


# Similar to part 1, I wrote out each of these functions myself then used chatgpt to help me refine them and identify inital small errors that I had
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define headers for web scraping
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# Define headers for Tasty API
TASTY_API_HEADERS = {
    "x-rapidapi-key": "10bff72081msh0d8b0bbdb700e4ap1d07bfjsn04ddcf3e4d60",
    "x-rapidapi-host": "tasty.p.rapidapi.com"
}

# USING very similar method as part 1, used copy and paste to start, but now making it into a function
def get_weekly_plan(meal_plan_number):
    """
    Scrapes a weekly meal plan from the tastesbetterfromscratch website.

    Argument:
        meal_plan_number (int): The number of the weekly meal plan (100-210).

    Returns:
        pandas.DataFrame: A DataFrame containing the meal plan data,
                          or an empty DataFrame if scraping fails.
    """
    url = f"https://tastesbetterfromscratch.com/meal-plan-{meal_plan_number}/"
    req = requests.Session()

    try:
        response = req.get(url, headers=HEADERS)
        response.raise_for_status()  # Raise an exception for bad status codes
        soup = BeautifulSoup(response.content, "html.parser")

        meal_plan_data = []
        main_content = soup.find('div', class_='site-inner')

        if main_content:
            meal_plan_items = main_content.find_all('p', class_='has-text-align-left')

            for item in meal_plan_items:
                text = item.get_text(strip=True)
                parts = text.split(':')
                if len(parts) >= 2:
                    day_of_week = parts[0].strip()
                    recipe_info = parts[1].strip()

                    name_and_price_parts = recipe_info.rsplit('$', 1)
                    if len(name_and_price_parts) == 2:
                        name_of_recipe = name_and_price_parts[0].strip()
                        price_of_recipe = '$' + name_and_price_parts[1].strip()
                    else:
                        name_of_recipe = recipe_info.strip()
                        price_of_recipe = 'N/A'

                    link_to_recipe = 'N/A'
                    link_tag = item.find('a')
                    if link_tag and link_tag.get('href'):
                         link_to_recipe = link_tag.get('href')

                    meal_plan_data.append({
                        'Day of the Week': day_of_week,
                        'Name of Recipe': name_of_recipe,
                        'Link to Recipe': link_to_recipe,
                        'Price of Recipe': price_of_recipe
                    })
        else:
            print(f"Could not find main content for meal plan {meal_plan_number}")
            return pd.DataFrame()


        return pd.DataFrame(meal_plan_data, columns=['Day of the Week', 'Name of Recipe', 'Link to Recipe', 'Price of Recipe'])

    except requests.exceptions.RequestException as e:
        print(f"Error fetching meal plan {meal_plan_number}: {e}")
        return pd.DataFrame()


get_weekly_plan(210)

,Day of the Week,Name of Recipe,Link to Recipe,Price of Recipe
0,Monday,Biscuit Chicken Pot Pie,https://tastesbetterfromscratch.com/biscuit-ch...,$11
1,Tuesday,Calzone,https://tastesbetterfromscratch.com/calzones/,$8
2,Wednesday,Quinoa Burger,https://tastesbetterfromscratch.com/quinoa-veg...,$15
3,Thursday,Short Rib Ragu,https://tastesbetterfromscratch.com/short-rib-...,$26
4,Friday,Cabbage Roll Soup,https://tastesbetterfromscratch.com/cabbage-ro...,$20


SO I know you asked for all the data and I originally did that, but by just selecting the things I want it made it so much easier for part 6 and for just seeing my data at each step that it was correct

If there was other questions that required other aspects, I could of gone back and changed the code to pull it (as I orignally did, but data got too big and messy)

In [7]:


# Defining headers for web scraping and headers for Tasty API
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

TASTY_API_HEADERS = {
    "x-rapidapi-key": "10bff72081msh0d8b0bbdb700e4ap1d07bfjsn04ddcf3e4d60",
    "x-rapidapi-host": "tasty.p.rapidapi.com"
}

#Again started by copying and pasting part 2 then making the edits to make it a function
def match_recipe(recipe_name):
    """
    Queries the Tasty API for recipes matching a given name.

    Argument:
        recipe_name (str): The name of the recipe to search for.

    Returns:
        pandas.DataFrame: A DataFrame containing up to 100 matching recipes
                          with key details (name, link, calories, etc.).
    """
    url = "https://tasty.p.rapidapi.com/recipes/list"
    querystring = {"from": "0", "size": "20", "q": recipe_name}

    try:
        response = requests.get(url, headers=TASTY_API_HEADERS, params=querystring)
        response.raise_for_status()
        data = response.json()

        if not data.get("results"):
            return pd.DataFrame()

        # Flatten JSON results
        all_recipes = pd.json_normalize(data, "results")

        # I am choosing only to keep these
        columns_to_keep = [
            "name",
            "slug",
            "user_ratings.score",
            "user_ratings.count_positive",
            "nutrition.calories",
            "nutrition.protein",
            "nutrition.fat",
            "nutrition.carbohydrates",
        ]

        # Only keep columns that actually exist in the data
        existing_cols = [c for c in columns_to_keep if c in all_recipes.columns]
        all_recipes = all_recipes[existing_cols].copy()

        # Create clickable link to Tasty recipe
        if "slug" in all_recipes.columns:
            all_recipes["link"] = "https://tasty.co/recipe/" + all_recipes["slug"]

        # Limit to first 100 recipes if needed
        return all_recipes.head(100)

    except requests.exceptions.RequestException as e:
        print(f"Error querying Tasty API for '{recipe_name}': {e}")
        return pd.DataFrame()

match_recipe('Chicken Tinga')

,name,slug,user_ratings.score,user_ratings.count_positive,nutrition.calories,nutrition.protein,nutrition.fat,nutrition.carbohydrates,link
0,Chicken Tinga Tacos (Tinga De Pollo),chicken-tinga-tacos-tinga-de-pollo,0.910112,243,653.0,39.0,33.0,56.0,https://tasty.co/recipe/chicken-tinga-tacos-ti...
1,Tostadas De Tinga,tostadas-de-tinga,1.000000,9,133.0,11.0,2.0,16.0,https://tasty.co/recipe/tostadas-de-tinga
2,Chicken Tinga Sheet Pan Quesadilla,chicken-tinga-sheet-pan-quesadilla,0.764706,13,390.0,20.0,18.0,31.0,https://tasty.co/recipe/chicken-tinga-sheet-pa...
3,Tostadas De Tinga De Pollo,tostadas-de-tinga-de-pollo,0.916667,11,287.0,27.0,16.0,10.0,https://tasty.co/recipe/tostadas-de-tinga-de-p...
4,Skillet Chicken Tinga Enchiladas,skillet-chicken-tinga-enchiladas,0.900000,9,1219.0,66.0,72.0,75.0,https://tasty.co/recipe/skillet-chicken-tinga-...
5,Bella’s Chicken Tinga Party Board,bellas-chicken-tinga-party-board,1.000000,4,52.0,7.0,2.0,0.0,https://tasty.co/recipe/bellas-chicken-tinga-p...
6,Chicken Tinga Loaded Nachos,chicken-tinga-loaded-nachos,1.000000,1,NaN,NaN,NaN,NaN,https://tasty.co/recipe/chicken-tinga-loaded-n...


In [8]:
import pandas as pd

def get_mealplan_data(meal_plan_number: int) -> pd.DataFrame:
    """
    Combine the scraped weekly meal plan and Tasty API matches
    for each day into a single dataset.

    Returns one row per *Tasty match*. If a day has no matches, we keep
    a single row with Tasty columns empty.
    """
    # 1) Scrape the weekly plan
    df_plan = get_weekly_plan(meal_plan_number)
    if df_plan.empty:
        # Return an empty frame with fallowing setup -- this is what I chose I wanted
        return pd.DataFrame(columns=[
            "Meal Plan Number", "Day of the Week", "Meal Plan Recipe Name",
            "Meal Plan Link", "Meal Plan Price",
            "Tasty Name", "Tasty Link", "Tasty Rating Score",
            "Tasty Rating Count Positive", "Calories", "Protein (g)",
            "Fat (g)", "Carbs (g)"
        ])

    out_frames = []

    # 2) For each day in the plan, query Tasty and assemble rows
    for _, row in df_plan.iterrows():
        day = row["Day of the Week"]
        recipe_name = row["Name of Recipe"]
        plan_link = row["Link to Recipe"]
        plan_price = row["Price of Recipe"]

        matches = match_recipe(recipe_name)  # your existing function

        if matches is None or matches.empty:
            # Keep one row with blanks for Tasty fields -- only keeping these rows because it is what is helpful for 6
            out_frames.append(pd.DataFrame([{
                "Meal Plan Number": meal_plan_number,
                "Day of the Week": day,
                "Meal Plan Recipe Name": recipe_name,
                "Meal Plan Link": plan_link,
                "Meal Plan Price": plan_price,
                "Tasty Name": pd.NA,
                "Tasty Link": pd.NA,
                "Tasty Rating Score": pd.NA,
                "Tasty Rating Count Positive": pd.NA,
                "Calories": pd.NA,
                "Protein (g)": pd.NA,
                "Fat (g)": pd.NA,
                "Carbs (g)": pd.NA,
            }]))
        else:
            # Keep only useful columns if present, then rename for clarity
            rename_map = {
                "name": "Tasty Name",
                "slug": "slug",  # temporary; will become link
                "user_ratings.score": "Tasty Rating Score",
                "user_ratings.count_positive": "Tasty Rating Count Positive",
                "nutrition.calories": "Calories",
                "nutrition.protein": "Protein (g)",
                "nutrition.fat": "Fat (g)",
                "nutrition.carbohydrates": "Carbs (g)",
            }

            keep_cols = [c for c in rename_map.keys() if c in matches.columns]
            tmp = matches[keep_cols].rename(columns=rename_map).copy()

            # Build the Tasty link if we have the slug
            if "slug" in tmp.columns:
                tmp["Tasty Link"] = "https://tasty.co/recipe/" + tmp["slug"]
                tmp.drop(columns=["slug"], inplace=True, errors="ignore")
            else:
                tmp["Tasty Link"] = pd.NA

            # Add the meal-plan context columns
            tmp.insert(0, "Meal Plan Number", meal_plan_number)
            tmp.insert(1, "Day of the Week", day)
            tmp.insert(2, "Meal Plan Recipe Name", recipe_name)
            tmp.insert(3, "Meal Plan Link", plan_link)
            tmp.insert(4, "Meal Plan Price", plan_price)

            # Ensure all expected columns exist
            for col in ["Tasty Name","Tasty Link","Tasty Rating Score",
                        "Tasty Rating Count Positive","Calories","Protein (g)",
                        "Fat (g)","Carbs (g)"]:
                if col not in tmp.columns:
                    tmp[col] = pd.NA

            out_frames.append(tmp)

    # 3) Combine all days; order columns nicely
    df = pd.concat(out_frames, ignore_index=True)

    final_cols = [
        "Meal Plan Number", "Day of the Week", "Meal Plan Recipe Name",
        "Meal Plan Link", "Meal Plan Price",
        "Tasty Name", "Tasty Link", "Tasty Rating Score",
        "Tasty Rating Count Positive", "Calories", "Protein (g)",
        "Fat (g)", "Carbs (g)"
    ]
    # Keep only final columns (some may be missing if no matches at all)
    for col in final_cols:
        if col not in df.columns:
            df[col] = pd.NA
    df = df[final_cols]

    return df


In [10]:
df = get_mealplan_data(202)
print(df.head(10).to_string(index=False))

 Meal Plan Number Day of the Week Meal Plan Recipe Name                                              Meal Plan Link Meal Plan Price                                 Tasty Name                                                       Tasty Link  Tasty Rating Score  Tasty Rating Count Positive  Calories  Protein (g)  Fat (g)  Carbs (g)
              202          Monday         Chicken Divan   https://tastesbetterfromscratch.com/skinny-chicken-divan/          $10.82                                        NaN                                                              NaN                 NaN                          NaN       NaN          NaN      NaN        NaN
              202         Tuesday      Beef Noodle Soup       https://tastesbetterfromscratch.com/beef-noodle-soup/          $29.29                 Taiwanese Beef Noodle Soup               https://tasty.co/recipe/taiwanese-beef-noodle-soup            0.982692                        511.0     887.0         68.0     33.0       81.0
    

/tmp/ipython-input-1202066621.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(out_frames, ignore_index=True)


# **Part 4**

In [11]:
# Defining a list of common meat names as you said we could do in the instructions
meat_names = ['chicken', 'beef', 'pork', 'lamb', 'turkey', 'bacon', 'ham', 'sausage', 'meatball']

# Function to check if a recipe name contains any meat name (case-insensitive)
def is_vegetarian(recipe_name):
    if pd.isna(recipe_name):
        return False
    recipe_name_lower = recipe_name.lower()
    for meat in meat_names:
        if meat in recipe_name_lower:
            return False
    return True

# Apply the function to the 'Meal Plan Recipe Name' column to create the new 'Is Vegetarian' column
df['Is Vegetarian'] = df['Meal Plan Recipe Name'].apply(is_vegetarian)

# Display the updated DataFrame with the new column
display(df)

,Meal Plan Number,Day of the Week,Meal Plan Recipe Name,Meal Plan Link,Meal Plan Price,Tasty Name,Tasty Link,Tasty Rating Score,Tasty Rating Count Positive,Calories,Protein (g),Fat (g),Carbs (g),Is Vegetarian
0,202,Monday,Chicken Divan,https://tastesbetterfromscratch.com/skinny-chi...,$10.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,202,Tuesday,Beef Noodle Soup,https://tastesbetterfromscratch.com/beef-noodl...,$29.29,Taiwanese Beef Noodle Soup,https://tasty.co/recipe/taiwanese-beef-noodle-...,0.982692,511.0,887.0,68.0,33.0,81.0,False
2,202,Tuesday,Beef Noodle Soup,https://tastesbetterfromscratch.com/beef-noodl...,$29.29,Classic Lasagna Soup,https://tasty.co/recipe/classic-lasagna-soup,0.954939,551.0,697.0,50.0,31.0,44.0,False
3,202,Tuesday,Beef Noodle Soup,https://tastesbetterfromscratch.com/beef-noodl...,$29.29,Traditional Vietnamese Beef Pho,https://tasty.co/recipe/traditional-vietnamese...,0.906114,415.0,539.0,27.0,20.0,59.0,False
4,202,Tuesday,Beef Noodle Soup,https://tastesbetterfromscratch.com/beef-noodl...,$29.29,1-Day Noodles (Taiwanese Beef Noodle Soup),https://tasty.co/recipe/1-day-noodles-taiwanes...,0.929825,53.0,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,202,Friday,Chicken Gyros,https://tastesbetterfromscratch.com/chicken-gy...,$21.22,5 Fork-less Finger Foods,https://tasty.co/recipe/5-fork-less-finger-foods,NaN,NaN,NaN,NaN,NaN,NaN,False
58,202,Friday,Chicken Gyros,https://tastesbetterfromscratch.com/chicken-gy...,$21.22,Homemade American Street Foods,https://tasty.co/recipe/homemade-american-stre...,NaN,NaN,NaN,NaN,NaN,NaN,False
59,202,Friday,Chicken Gyros,https://tastesbetterfromscratch.com/chicken-gy...,$21.22,Chicken Recipes For The Whole Month,https://tasty.co/recipe/chicken-recipes-for-th...,NaN,NaN,NaN,NaN,NaN,NaN,False
60,202,Friday,Chicken Gyros,https://tastesbetterfromscratch.com/chicken-gy...,$21.22,Tasty Dinners for Every Night of December,https://tasty.co/recipe/tasty-dinner-for-every...,NaN,NaN,NaN,NaN,NaN,NaN,False


# **Part 6**

In [12]:
import altair as alt
import pandas as pd

# Select relevant columns for visualization, excluding Calories as requested
df_viz = df[[
    "Day of the Week", "Meal Plan Recipe Name", "Tasty Name", "Is Vegetarian",
    "Protein (g)", "Fat (g)", "Carbs (g)"
]].copy()

# Melt the dataframe to long format for Altair
df_melted = df_viz.melt(
    id_vars=["Day of the Week", "Meal Plan Recipe Name", "Tasty Name", "Is Vegetarian"],
    var_name="Nutrition Type",
    value_name="Amount"
)

# Drop rows where 'Amount' is NaN
df_melted.dropna(subset=["Amount"], inplace=True)

# Calculate the average amount for each nutrition type, grouped by day and vegetarian status
df_avg_nutrition_day = df_melted.groupby(["Day of the Week", "Nutrition Type", "Is Vegetarian"])["Amount"].mean().reset_index()

# Define the order of days for the x-axis
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Create the Altair chart for average nutrition per day
chart = alt.Chart(df_avg_nutrition_day).mark_bar().encode(
    x=alt.X("Nutrition Type", axis=None), # Hide x-axis labels for Nutrition Type
    y=alt.Y("Amount", title="Average Amount (g)"),
    color=alt.Color("Is Vegetarian", legend=alt.Legend(title="Vegetarian")),
    column=alt.Column("Day of the Week", sort=day_order, header=alt.Header(titleOrient="bottom", labelOrient="bottom")),
    tooltip=[
        "Day of the Week",
        "Nutrition Type",
        "Amount",
        "Is Vegetarian"
    ]
).properties(
    title="Average Protein, Fat, and Carbs per Day (Meal Plan 202)"
).interactive() # Make the chart interactive

chart.display()

alt.Chart(...)

This looks at the average amount of carbs, fat, and protien for each of the weeks Meal Types, finding that average by taking the average of cabrs, fats, and protein for all of the Tasty meals that fit under that category. Note there is no Monday as nothing matched Chicken Divan in the Tasty webscraping.

The biggest thing I noticed is the amount of protein, which by hovering the mouse over it we can see as the third bar. There is a LOT more protein in the non-vegetarian days than vegetarian. This makes a lot of sense as meat is obviously a huge source of protein.

Other than that, there doesn't seem to be a big differnce in carbs and fat in vegetarian and non-vegitarian, but natrually there is some variation.

There also appears to be less Fat throughout the week which is an intresting oberservation!